In [ ]:
"""
Anveshak: Spirituality Q&A - Data Preprocessing Pipeline

This script processes the spiritual text corpus for the Anveshak application:
1. Uploads and downloads text files from various sources
2. Cleans and processes the texts to remove artifacts and noise
3. Chunks texts into smaller, manageable pieces
4. Generates embeddings using the E5-large-v2 model
5. Creates a FAISS index for efficient similarity search
6. Uploads all processed data to Google Cloud Storage

Usage:
- Run in Google Colab with GPU runtime for faster embedding generation
- Ensure GCP authentication is set up before running
- Configure the constants below with your actual settings
"""

In [ ]:
# =============================================================================
# CONFIGURATION SETTINGS
# =============================================================================
# Update these values with your actual settings
# Before open-sourcing, clear these values or replace with placeholders
BUCKET_NAME_GCS = "your-bucket-name"  # e.g., "spiritual-texts-bucket"
EMBEDDING_MODEL = "your-embedding-model"  # e.g., "intfloat/e5-large-v2"
# LLM_MODEL = "your-llm-model"  # e.g., "gpt-3.5-turbo"

# GCS Paths - update these with your folder structure
METADATA_PATH_GCS = "metadata/metadata.jsonl"
RAW_TEXTS_UPLOADED_PATH_GCS = "raw-texts/uploaded"
RAW_TEXTS_DOWNLOADED_PATH_GCS = "raw-texts/downloaded/"
CLEANED_TEXTS_PATH_GCS = "cleaned-texts/"
EMBEDDINGS_PATH_GCS = "processed/embeddings/all_embeddings.npy"
INDICES_PATH_GCS = "processed/indices/faiss_index.faiss"
CHUNKS_PATH_GCS = "processed/chunks/text_chunks.txt"

# Local file paths in Colab environment - update these with your folder structure
LOCAL_METADATA_FILE = "/content/metadata.jsonl"
LOCAL_RAW_TEXTS_FOLDER = "/content/raw-texts/uploaded"
LOCAL_EMBEDDINGS_FILE = "/tmp/all_embeddings.npy"
LOCAL_FAISS_INDEX_FILE = "/tmp/faiss_index.faiss"
LOCAL_TEXT_CHUNKS_FILE = "/tmp/text_chunks.txt"

In [ ]:
# Install required packages
!pip install faiss-cpu

In [ ]:
# Import necessary libraries
from google.colab import files
from google.colab import auth
from google.cloud import storage
import os
import json
import requests
import re
import unicodedata
from bs4 import BeautifulSoup
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
# =============================================================================
# AUTHENTICATION & INITIALIZATION
# =============================================================================

# Authenticate with Google Cloud (only needed in Colab)
auth.authenticate_user()

# Initialize GCS client (single initialization)
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME_GCS)

In [ ]:
# =============================================================================
# PART 1: UPLOAD RAW TEXTS AND METADATA
# =============================================================================

def upload_files_to_colab():
    """
    Upload raw text files and metadata from local machine to Colab.

    This function:
    1. Prompts the user to upload text files
    2. Saves the uploaded files to a local directory
    3. Prompts the user to upload the metadata.jsonl file
    4. Saves the metadata file to the specified location

    Returns:
        bool: True if upload was successful, False otherwise
    """
    # First, upload text files
    print("Step 1: Please upload your text files...")
    uploaded_text_files = files.upload()  # This will prompt the user to upload files

    # Create directory structure if it doesn't exist
    os.makedirs(LOCAL_RAW_TEXTS_FOLDER, exist_ok=True)

    # Move uploaded text files to the raw-texts folder
    for filename, content in uploaded_text_files.items():
        if filename.endswith(".txt"):
            with open(os.path.join(LOCAL_RAW_TEXTS_FOLDER, filename), "wb") as f:
                f.write(content)
            print(f"✅ Saved {filename} to {LOCAL_RAW_TEXTS_FOLDER}")

    print("Text files upload complete!")

    # Next, upload metadata file
    print("\nStep 2: Please upload your metadata.jsonl file...")
    uploaded_metadata = files.upload()  # This will prompt the user to upload files

    # Save metadata file
    metadata_uploaded = False
    for filename, content in uploaded_metadata.items():
        if filename == "metadata.jsonl":
            # Ensure the directory for metadata file exists
            os.makedirs(os.path.dirname(LOCAL_METADATA_FILE), exist_ok=True)
            with open(LOCAL_METADATA_FILE, "wb") as f:
                f.write(content)
            print(f"✅ Saved metadata.jsonl to {LOCAL_METADATA_FILE}")
            metadata_uploaded = True

    if not metadata_uploaded:
        print("⚠️ Warning: metadata.jsonl was not uploaded. Please upload it to continue.")
        return False

    print("Upload to Colab complete!")
    return True

def upload_files_to_gcs():
    """
    Upload raw text files and metadata from Colab to Google Cloud Storage.

    This function:
    1. Uploads each text file from the local directory to GCS
    2. Uploads the metadata.jsonl file to GCS

    All files are uploaded to the paths specified in the configuration constants.
    """
    # Upload each file from the local raw-texts folder to GCS
    for filename in os.listdir(LOCAL_RAW_TEXTS_FOLDER):
        local_path = os.path.join(LOCAL_RAW_TEXTS_FOLDER, filename)
        blob_path = f"{RAW_TEXTS_UPLOADED_PATH_GCS}/{filename}"  # GCS path
        blob = bucket.blob(blob_path)
        try:
            blob.upload_from_filename(local_path)
            print(f"✅ Uploaded: {filename} -> gs://{BUCKET_NAME_GCS}/{blob_path}")
        except Exception as e:
            print(f"❌ Failed to upload {filename}: {e}")

    # Upload metadata file
    blob = bucket.blob(METADATA_PATH_GCS)
    try:
        blob.upload_from_filename(LOCAL_METADATA_FILE)
        print(f"✅ Uploaded metadata.jsonl -> gs://{BUCKET_NAME_GCS}/{METADATA_PATH_GCS}")
    except Exception as e:
        print(f"❌ Failed to upload metadata: {e}")

In [ ]:
# =============================================================================
# PART 2: DOWNLOAD AND CLEAN TEXTS
# =============================================================================

def fetch_metadata_from_gcs():
    """
    Fetch metadata.jsonl from GCS and return as a list of dictionaries.

    Each dictionary represents a text entry with metadata like title, author, etc.

    Returns:
        list: List of dictionaries containing metadata for each text
    """
    blob = bucket.blob(METADATA_PATH_GCS)
    # Download metadata file
    metadata_jsonl = blob.download_as_text()
    # Parse JSONL
    metadata = [json.loads(line) for line in metadata_jsonl.splitlines()]
    return metadata

def upload_to_gcs(source_file, destination_path):
    """
    Upload a local file to Google Cloud Storage.

    Args:
        source_file (str): Path to the local file
        destination_path (str): Path in GCS where the file should be uploaded
    """
    blob = bucket.blob(destination_path)
    blob.upload_from_filename(source_file)
    print(f"📤 Uploaded to GCS: {destination_path}")

def download_text_files():
    """
    Download text files from URLs specified in the metadata.

    This function:
    1. Fetches metadata from GCS
    2. Filters entries where Uploaded=False (texts to be downloaded)
    3. Downloads each text from its URL
    4. Uploads the downloaded text to GCS

    This allows automated collection of texts that weren't manually uploaded.
    """
    metadata = fetch_metadata_from_gcs()
    # Filter entries where Uploaded is False
    files_to_download = [item for item in metadata if item["Uploaded"] == False]
    print(f"🔍 Found {len(files_to_download)} files to download")

    # Process only necessary files
    for item in files_to_download:
        name, author, url = item["Title"], item["Author"], item["URL"]
        if url.lower() == "not available":
            print(f"❌ Skipping {name} - No URL available.")
            continue

        try:
            response = requests.get(url)
            if response.status_code == 200:
                raw_text = response.text
                filename = "{}.txt".format(name.replace(" ", "_"))
                # Save to local first
                local_path = f"/tmp/{filename}"
                with open(local_path, "w", encoding="utf-8") as file:
                    file.write(raw_text)
                # Upload to GCS
                gcs_path = f"{RAW_TEXTS_DOWNLOADED_PATH_GCS}{filename}"
                upload_to_gcs(local_path, gcs_path)
                print(f"✅ Downloaded & uploaded: {filename} ({len(raw_text.split())} words)")
                # Clean up temp file
                os.remove(local_path)
            else:
                print(f"❌ Failed to download {name}: {url} (Status {response.status_code})")
        except Exception as e:
            print(f"❌ Error processing {name}: {e}")

def rigorous_clean_text(text):
    """
    Clean text by removing metadata, junk text, and formatting issues.

    This function:
    1. Removes HTML tags using BeautifulSoup
    2. Removes URLs and standalone numbers
    3. Removes all-caps OCR noise words
    4. Deduplicates adjacent identical lines
    5. Normalizes Unicode characters
    6. Standardizes whitespace and newlines

    Args:
        text (str): The raw text to clean

    Returns:
        str: The cleaned text
    """
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"https?:\/\/\S+", "", text)  # Remove links
    text = re.sub(r"\b\d+\b", "", text)  # Remove standalone numbers
    text = re.sub(r"\b[A-Z]{5,}\b", "", text)  # Remove all-caps OCR noise words
    lines = text.split("\n")
    cleaned_lines = []
    last_line = None

    for line in lines:
        line = line.strip()
        if line and line != last_line:
            cleaned_lines.append(line)
            last_line = line

    text = "\n".join(cleaned_lines)
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"\n{2,}", "\n", text)
    return text

def clean_and_upload_texts():
    """
    Download raw texts from GCS, clean them, and upload cleaned versions back to GCS.

    This function processes all texts in both the uploaded and downloaded folders:
    1. For each text file, downloads it from GCS
    2. Cleans the text using rigorous_clean_text()
    3. Uploads the cleaned version back to GCS in the cleaned-texts folder

    This step ensures that all texts are properly formatted before embedding generation.
    """
    raw_texts_folders = [RAW_TEXTS_DOWNLOADED_PATH_GCS, RAW_TEXTS_UPLOADED_PATH_GCS]  # Process both folders
    total_files = 0  # Counter to track number of processed files

    for raw_texts_folder in raw_texts_folders:
        # List all files in the current raw-texts folder
        blobs = list(bucket.list_blobs(prefix=raw_texts_folder))
        print(f"🔍 Found {len(blobs)} files in {raw_texts_folder}")

        for blob in blobs:
            if not blob.name.endswith(".txt"):  # Skip non-text files
                continue

            try:
                # Download file
                raw_text = blob.download_as_text().strip()
                if not raw_text:  # Skip empty files
                    print(f"⚠️ Skipping empty file: {blob.name}")
                    continue

                # Clean text
                cleaned_text = rigorous_clean_text(raw_text)

                # Save cleaned text back to GCS
                cleaned_blob_name = blob.name.replace(raw_texts_folder, CLEANED_TEXTS_PATH_GCS)
                cleaned_blob = bucket.blob(cleaned_blob_name)
                cleaned_blob.upload_from_string(cleaned_text, content_type="text/plain")
                print(f"✅ Cleaned & uploaded: {cleaned_blob_name} ({len(cleaned_text.split())} words, {len(cleaned_text)} characters)")
                total_files += 1
            except Exception as e:
                print(f"❌ Error processing {blob.name}: {e}")

    print(f"🚀 Cleaning process completed! Total cleaned & uploaded files: {total_files}")

In [ ]:
# =============================================================================
# PART 3: GENERATE EMBEDDINGS AND INDEX
# =============================================================================

def fetch_metadata_dict_from_gcs():
    """
    Fetch metadata.jsonl from GCS and return as a dictionary.

    The dictionary is keyed by title for easy lookup during text processing.

    Returns:
        dict: Dictionary mapping text titles to their metadata
    """
    metadata_blob = bucket.blob(METADATA_PATH_GCS)
    metadata_dict = {}

    if metadata_blob.exists():
        metadata_content = metadata_blob.download_as_text()
        for line in metadata_content.splitlines():
            item = json.loads(line)
            metadata_dict[item["Title"]] = item  # Keep space-based lookup
    else:
        print("❌ Metadata file not found in GCS")

    return metadata_dict

def chunk_text(text, chunk_size=500, overlap=50):
    """
    Split text into smaller, overlapping chunks for better retrieval.

    Args:
        text (str): The text to chunk
        chunk_size (int): Maximum number of words per chunk
        overlap (int): Number of words to overlap between chunks

    Returns:
        list: List of text chunks
    """
    words = text.split()
    chunks = []
    i = 0

    while i < len(words):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
        i += chunk_size - overlap

    return chunks

def create_embeddings(text_chunks, batch_size=32):
    """
    Generate embeddings for the given chunks of text using the specified embedding model.

    This function:
    1. Uses SentenceTransformer to load the embedding model
    2. Prefixes each chunk with "passage:" as required by the E5 model
    3. Processes chunks in batches to manage memory usage
    4. Normalizes embeddings for cosine similarity search

    Args:
        text_chunks (list): List of text chunks to embed
        batch_size (int): Number of chunks to process at once

    Returns:
        numpy.ndarray: Matrix of embeddings, one per text chunk
    """
    # Load the model with GPU optimization
    model = SentenceTransformer(EMBEDDING_MODEL)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    print(f"🚀 Using device for embeddings: {device}")

    prefixed_chunks = [f"passage: {text}" for text in text_chunks]
    all_embeddings = []

    for i in range(0, len(prefixed_chunks), batch_size):
        batch = prefixed_chunks[i:i+batch_size]

        # Move batch to GPU (if available) for faster processing
        with torch.no_grad():
            batch_embeddings = model.encode(batch, convert_to_numpy=True, normalize_embeddings=True)

        all_embeddings.append(batch_embeddings)

        if (i + batch_size) % 100 == 0 or (i + batch_size) >= len(prefixed_chunks):
            print(f"📌 Processed {i + min(batch_size, len(prefixed_chunks) - i)}/{len(prefixed_chunks)} documents")

    return np.vstack(all_embeddings).astype("float32")

def process_cleaned_texts():
    """
    Process cleaned texts to create embeddings, FAISS index, and text chunks with metadata.

    This function:
    1. Downloads all cleaned texts from GCS
    2. Chunks each text into smaller pieces
    3. Generates embeddings for each chunk
    4. Creates a FAISS index for similarity search
    5. Saves and uploads all processed data back to GCS

    This is the core processing step that prepares data for the RAG system.
    """
    all_chunks = []
    all_metadata = []
    chunk_counter = 0

    metadata_dict = fetch_metadata_dict_from_gcs()  # Load metadata

    # Optimized listing of blobs in cleaned-texts folder
    blobs = list(storage_client.list_blobs(BUCKET_NAME_GCS, prefix=CLEANED_TEXTS_PATH_GCS))
    print(f"🔍 Found {len(blobs)} files in {CLEANED_TEXTS_PATH_GCS}")

    if not blobs:
        print(f"❌ No files found in {CLEANED_TEXTS_PATH_GCS}. Exiting.")
        return

    for blob in blobs:
        file_name = blob.name.split("/")[-1]
        if not file_name or file_name.startswith("."):
            continue  # Skip empty or hidden files

        # Convert filename back to space-based title for metadata lookup
        book_name = file_name.replace("_", " ")
        metadata = metadata_dict.get(book_name, {"Author": "Unknown", "Publisher": "Unknown"})
        author = metadata.get("Author", "Unknown")

        try:
            # Download and read text
            raw_text = blob.download_as_text().strip()

            # Skip empty or corrupt files
            if not raw_text:
                print(f"❌ Skipping empty file: {file_name}")
                continue

            chunks = chunk_text(raw_text)
            print(f"✅ Processed {book_name}: {len(chunks)} chunks")

            for chunk in chunks:
                all_chunks.append(chunk)
                all_metadata.append((chunk_counter, book_name, author))
                chunk_counter += 1
        except Exception as e:
            print(f"❌ Error processing {file_name}: {e}")

    # Ensure there are chunks before embedding generation
    if not all_chunks:
        print("❌ No chunks found. Skipping embedding generation.")
        return

    # Create embeddings with GPU acceleration
    print(f"📍 Creating embeddings for {len(all_chunks)} total chunks...")
    all_embeddings = create_embeddings(all_chunks)

    # Build FAISS index
    dimension = all_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(all_embeddings)
    print(f"✅ FAISS index built with {index.ntotal} vectors")

    # Save & upload embeddings
    np.save(LOCAL_EMBEDDINGS_FILE, all_embeddings)  # Save locally first
    embeddings_blob = bucket.blob(EMBEDDINGS_PATH_GCS)
    embeddings_blob.upload_from_filename(LOCAL_EMBEDDINGS_FILE)
    print(f"✅ Uploaded embeddings to GCS: {EMBEDDINGS_PATH_GCS}")

    # Save & upload FAISS index
    faiss.write_index(index, LOCAL_FAISS_INDEX_FILE)
    index_blob = bucket.blob(INDICES_PATH_GCS)
    index_blob.upload_from_filename(LOCAL_FAISS_INDEX_FILE)
    print(f"✅ Uploaded FAISS index to GCS: {INDICES_PATH_GCS}")

    # Save and upload text chunks with metadata
    with open(LOCAL_TEXT_CHUNKS_FILE, "w", encoding="utf-8") as f:
        for i, (chunk_id, book_name, author) in enumerate(all_metadata):
            f.write(f"{i}\t{book_name}\t{author}\t{all_chunks[i]}\n")

    chunks_blob = bucket.blob(CHUNKS_PATH_GCS)
    chunks_blob.upload_from_filename(LOCAL_TEXT_CHUNKS_FILE)
    print(f"✅ Uploaded text chunks to GCS: {CHUNKS_PATH_GCS}")

    # Clean up temp files
    os.remove(LOCAL_EMBEDDINGS_FILE)
    os.remove(LOCAL_FAISS_INDEX_FILE)
    os.remove(LOCAL_TEXT_CHUNKS_FILE)

In [ ]:
# =============================================================================
# PART 4: MAIN EXECUTION
# =============================================================================

def run_pipeline():
    """
    Run the complete end-to-end preprocessing pipeline.

    This function executes all steps in sequence:
    1. Upload files from local to Colab
    2. Upload raw texts and metadata to GCS
    3. Download texts from URLs specified in metadata
    4. Clean and process all texts
    5. Generate embeddings and build the FAISS index

    This is the main entry point for the preprocessing script.
    """
    print("🚀 Starting pipeline execution...")

    print("\n==== STEP 1: Uploading files from local to Colab ====")
    upload_successful = upload_files_to_colab()

    if not upload_successful:
        print("❌ Pipeline halted due to missing metadata file.")
        return

    print("\n==== STEP 2: Uploading raw texts and metadata to GCS ====")
    upload_files_to_gcs()

    print("\n==== STEP 3: Downloading texts from URLs ====")
    download_text_files()

    print("\n==== STEP 4: Cleaning and processing texts ====")
    clean_and_upload_texts()

    print("\n==== STEP 5: Generating embeddings and building index ====")
    process_cleaned_texts()

    print("\n✅ Pipeline execution completed successfully!")

# Execute the complete pipeline
if __name__ == "__main__":
    run_pipeline()